# Running Time-Series Simulations with FEEMS

This notebook demonstrates how to run **realistic time-series simulations** using the system created in the basic example.

## What You'll Learn

1. **Loading saved systems**: Import system models from protobuf files
2. **RunFeemsSim package**: Higher-level simulation interface
3. **Time-series profiles**: Realistic operational scenarios
4. **MachineryCalculation**: Automated power management
5. **Results analysis**: Comprehensive performance metrics

## Prerequisites

⚠️ **Important**: Run `00_Basic_Example.ipynb` first to create the system model file.

## Use Case

We'll simulate a vessel operating over a 1000-second period with:
- Variable propulsion power (simulating speed changes, maneuvering)
- Variable auxiliary power (hotel loads fluctuating)
- Automatic generator management
- Continuous fuel and emission tracking

## 1. Load the System Model

We'll load the electric power system that was saved as a protobuf file in the basic example.

**Why protobuf?**
- Standardized format for data exchange
- Compact binary representation
- Language-independent (can be used in C++, Python, Java, etc.)
- Suitable for archiving and version control

In [1]:
# Load the system model from protobuf file
import os
from MachSysS.system_structure_pb2 import MachinerySystem
from MachSysS.convert_to_feems import convert_proto_propulsion_system_to_feems

file_path = os.path.join("data", "electric_power_system.pb")
if not os.path.exists(file_path):
    raise FileNotFoundError(
        f"Model file not found at {file_path}. "
        f"Please run `00_Basic_Example.ipynb` first to create the system model."
    )

# Load the protobuf file
print(f"Loading system from: {file_path}")
with open(file_path, "rb") as f:
    system = MachinerySystem()
    system.ParseFromString(f.read())

# Convert the protobuf system to FEEMS format
feems_system = convert_proto_propulsion_system_to_feems(system)

print(f"✓ System loaded: {feems_system.name}")
print(f"  Power sources: {feems_system.no_power_sources}")
print(f"  Propulsion units: {feems_system.no_propulsion_units}")
print(f"  Other loads: {feems_system.no_other_load}")
print(f"  Switchboards: {feems_system.no_switchboard}")

Loading system from: data/electric_power_system.pb
✓ System loaded: electric power system
  Power sources: 2
  Propulsion units: 2
  Other loads: 1
  Switchboards: 2


## 2. Setup and Run the Simulation

### Create Time-Series Load Profiles

We'll generate realistic operational profiles:

**Propulsion loads**:
- Random variations simulating speed changes and maneuvering
- Separate profiles for each thruster (different rated powers)

**Auxiliary loads**:
- Random variations simulating hotel load fluctuations
- HVAC cycling, galley operations, etc.

### MachineryCalculation

`MachineryCalculation` is a high-level interface that:
- Accepts time-series power demands
- Automatically manages generator start/stop (if PMS is configured)
- Performs power balance at each time step
- Tracks fuel consumption and emissions
- Returns comprehensive results


In [2]:

from pprint import pprint
import numpy as np
import pandas as pd
from RunFeemsSim.machinery_calculation import MachineryCalculation
from feems.fuel import FuelSpecifiedBy

# Initialize the machinery calculation
machinery_calculation = MachineryCalculation(feems_system)

# Simulation parameters
number_points_load = 1000  # 1000 time points
time_interval_s = 1.0      # 1 second intervals (1000 seconds total)

# Get rated powers for the propulsion drives
rated_power_for_propulsion_drive_1 = feems_system.propulsion_drives[0].rated_power
rated_power_for_propulsion_drive_2 = feems_system.propulsion_drives[1].rated_power

print(f"Simulation setup:")
print(f"  Duration: {number_points_load * time_interval_s} seconds")
print(f"  Time step: {time_interval_s} seconds")
print(f"  Propulsion drive 1 rated power: {rated_power_for_propulsion_drive_1} kW")
print(f"  Propulsion drive 2 rated power: {rated_power_for_propulsion_drive_2} kW")

# Create propulsion power time series
# Random power between 0 and rated power for each drive
propulsion_power_ts = pd.DataFrame(
    index=np.arange(0, number_points_load * time_interval_s, time_interval_s),
    data={
        feems_system.propulsion_drives[0].name: 
            rated_power_for_propulsion_drive_1 * np.random.random(number_points_load),
        feems_system.propulsion_drives[1].name: 
            rated_power_for_propulsion_drive_2 * np.random.random(number_points_load),
    }
)

# Create auxiliary power time series (hotel loads)
# Random power between 0 and 100 kW
auxiliary_load = 100 * np.random.random(number_points_load)

print(f"\nLoad profiles generated:")
print(f"  Propulsion 1: {propulsion_power_ts.iloc[:, 0].mean():.1f} kW (average)")
print(f"  Propulsion 2: {propulsion_power_ts.iloc[:, 1].mean():.1f} kW (average)")
print(f"  Auxiliary: {auxiliary_load.mean():.1f} kW (average)")

# Run the simulation
print(f"\nRunning simulation...")
result = machinery_calculation.calculate_machinery_system_output_from_propulsion_power_time_series(
    propulsion_power=propulsion_power_ts,
    auxiliary_power_kw=auxiliary_load,
    fuel_specified_by=FuelSpecifiedBy.FUEL_EU_MARITIME,  # Use FuelEU Maritime emission factors
)

print(f"✓ Simulation complete!\n")

# Print the full results dictionary
print("="*70)
print("DETAILED SIMULATION RESULTS")
print("="*70)
pprint(result.__dict__)

Simulation setup:
  Duration: 1000.0 seconds
  Time step: 1.0 seconds
  Propulsion drive 1 rated power: 800.0 kW
  Propulsion drive 2 rated power: 600.0 kW

Load profiles generated:
  Propulsion 1: 398.2 kW (average)
  Propulsion 2: 299.8 kW (average)
  Auxiliary: 50.1 kW (average)

Running simulation...
✓ Simulation complete!

DETAILED SIMULATION RESULTS
{'co2_emission_total_kg': GHGEmissions(tank_to_wake_kg_or_gco2eq_per_gfuel=np.float64(161.72999103297047),
                                       well_to_tank_kg_or_gco2eq_per_gfuel=np.float64(30.53733832634306),
                                       tank_to_wake_kg_or_gco2eq_per_gfuel_without_slip=np.float64(159.2224607634918),
                                       tank_to_wake_kg_or_gco2eq_per_gfuel_from_green_fuel=np.float64(0.0),
                                       tank_to_wake_kg_or_gco2eq_per_gfuel_without_slip_from_green_fuel=np.float64(0.0)),
 'detail_result':                                 multi fuel consumption [kg]  \

## 3. Analyze the Results

The simulation returns comprehensive metrics. Let's break them down:

### Fuel Consumption

Total fuel consumed across all generators during the simulation period.

print("\n" + "="*70)
print("FUEL CONSUMPTION")
print("="*70)
for fuel_consumption in result.multi_fuel_consumption_total_kg.fuels:
    print(f"\n{fuel_consumption.fuel_type.name}:")
    print(f"  Total consumed: {fuel_consumption.mass_or_mass_fraction:.2f} kg")
    print(f"  Duration: {result.duration_s:.0f} seconds ({result.duration_s/3600:.2f} hours)")
    print(f"  Consumption rate: {fuel_consumption.mass_or_mass_fraction/(result.duration_s/3600):.2f} kg/hr")

### GHG Emissions

Greenhouse gas emissions calculated according to FuelEU Maritime methodology:
- **Well-to-tank**: Emissions from fuel production and transportation
- **Tank-to-wake**: Emissions from fuel combustion (vessel operation)
- **Well-to-wake**: Total lifecycle emissions


In [3]:

print("\n" + "="*70)
print("GHG EMISSIONS (FuelEU Maritime)")
print("="*70)
print(f"\n  Well-to-tank: {result.co2_emission_total_kg.well_to_tank_kg_or_gco2eq_per_gfuel:.2f} kg CO2eq")
print(f"    (Upstream emissions from fuel production)")
print(f"\n  Tank-to-wake: {result.co2_emission_total_kg.tank_to_wake_emissions_kg_for_ets:.2f} kg CO2eq")
print(f"    (Operational emissions - counted for EU ETS)")
print(f"\n  Well-to-wake: {result.co2_emission_total_kg.well_to_wake_kg_or_gco2eq_per_gfuel:.2f} kg CO2eq")
print(f"    (Total lifecycle emissions)")

### Other Emissions

print("\n" + "="*70)
print("OTHER EMISSIONS")
print("="*70)
for emission_type, emission_kg in result.total_emission_kg.items():
    print(f"\n{emission_type.name}: {emission_kg:.3f} kg")

### Energy Consumption Breakdown

print("\n" + "="*70)
print("ENERGY CONSUMPTION")
print("="*70)
print(f"\nPropulsion energy: {result.energy_consumption_propulsion_total_mj:.2f} MJ")
print(f"  ({result.energy_consumption_propulsion_total_mj/3.6:.1f} kWh)")
print(f"\nAuxiliary energy: {result.energy_consumption_auxiliary_total_mj:.2f} MJ")
print(f"  ({result.energy_consumption_auxiliary_total_mj/3.6:.1f} kWh)")
print(f"\nTotal energy: {result.energy_consumption_propulsion_total_mj + result.energy_consumption_auxiliary_total_mj:.2f} MJ")
print(f"  ({(result.energy_consumption_propulsion_total_mj + result.energy_consumption_auxiliary_total_mj)/3.6:.1f} kWh)")

### Operating Hours

print("\n" + "="*70)
print("OPERATING HOURS")
print("="*70)
print(f"\nTotal genset running hours: {result.running_hours_genset_total_hr:.3f} hr")
print(f"  (Sum of all generator operating hours)")
print(f"\nSimulation duration: {result.duration_s/3600:.3f} hr")
print(f"Number of gensets: {feems_system.no_power_sources}")
print(f"Average genset utilization: {result.running_hours_genset_total_hr/(result.duration_s/3600*feems_system.no_power_sources)*100:.1f}%")

### Results per Component

# This table shows individual genset performance:

print("\n" + "="*70)
print("RESULTS PER COMPONENT")
print("="*70)
print(result.detail_result)


GHG EMISSIONS (FuelEU Maritime)

  Well-to-tank: 30.54 kg CO2eq
    (Upstream emissions from fuel production)

  Tank-to-wake: 161.73 kg CO2eq
    (Operational emissions - counted for EU ETS)

  Well-to-wake: 192.27 kg CO2eq
    (Total lifecycle emissions)

OTHER EMISSIONS

NOX: 2.097 kg

ENERGY CONSUMPTION

Propulsion energy: 697.65 MJ
  (193.8 kWh)

Auxiliary energy: 50.08 MJ
  (13.9 kWh)

Total energy: 747.73 MJ
  (207.7 kWh)

OPERATING HOURS

Total genset running hours: 0.453 hr
  (Sum of all generator operating hours)

Simulation duration: 0.278 hr
Number of gensets: 2
Average genset utilization: 81.6%

RESULTS PER COMPONENT
                                multi fuel consumption [kg]  \
Genset 1  FuelConsumption(fuels=[<feems.fuel.Fuel object...   
Genset 2  FuelConsumption(fuels=[<feems.fuel.Fuel object...   

         electric energy consumption [MJ] mechanical energy consumption [MJ]  \
Genset 1                                0                         354.986741   
Genset 2   

## 4. Key Insights from the Results

### Understanding the Metrics

**Fuel Consumption**: ~49 kg over 16.6 minutes
- Extrapolated hourly rate: ~177 kg/hr
- This represents both gensets running to supply propulsion and auxiliary loads

**CO2 Emissions**: ~160 kg tank-to-wake, ~190 kg well-to-wake
- Well-to-wake includes upstream production emissions (~30 kg)
- Important for FuelEU Maritime compliance and carbon accounting

**NOx Emissions**: ~2.07 kg
- Air quality concern in port areas and ECAs
- Subject to IMO Tier III regulations in emission control areas

**Energy Distribution**:
- Propulsion: 686 MJ (93%) - main power consumers
- Auxiliary: 50 MJ (7%) - hotel loads
- This ratio is typical for vessels in transit

**Genset Operating Hours**: 0.45 hr total
- Both gensets operating for ~27% of simulation time
- In practice, PMS logic would optimize genset start/stop

### Performance Indicators

The `detail_result` DataFrame provides per-component insights:
- **Load distribution**: Which genset carries more load?
- **Operating hours**: Runtime for each genset
- **Efficiency**: Fuel consumed per unit energy output

### Comparison to Basic Example

The basic example showed:
- **100 time points** with manual load configuration
- **Static** bus-tie configuration
- **Direct** FEEMS system interface

This simulation example shows:
- **1000 time points** for realistic operational profiles
- **Automated** power management through RunFeemsSim
- **Higher-level** interface suitable for route planning and performance prediction

## 5. Applications

This simulation approach is useful for:

### 1. Route Planning
- Estimate fuel consumption for planned voyages
- Optimize speed profiles for fuel efficiency
- Predict port-to-port energy requirements

### 2. Emissions Reporting
- IMO DCS (Data Collection System)
- EU MRV (Monitoring, Reporting, Verification)
- FuelEU Maritime compliance
- Carbon intensity indicators (CII)

### 3. System Design
- Size generators for operational profiles
- Evaluate hybrid/battery integration
- Compare different power plant configurations

### 4. Performance Benchmarking
- Compare actual vs. predicted performance
- Identify efficiency degradation
- Optimize operational procedures

## Summary

In this notebook, we learned:

### 1. System Loading
- Import systems from protobuf files
- MachSysS package for data interchange

### 2. Time-Series Simulation
- Define realistic load profiles
- Use pandas DataFrames for time-series data
- Run multi-point simulations efficiently

### 3. RunFeemsSim Interface
- `MachineryCalculation` class for automated simulation
- Higher-level API compared to direct FEEMS usage
- Suitable for integration into larger tools

### 4. Comprehensive Results
- Fuel consumption and emissions tracking
- Energy breakdown by consumer type
- Component-level performance metrics
- Regulatory compliance data (FuelEU Maritime)

### Next Steps

**Explore advanced features**:
- **Power Management Systems**: See RunFeemsSim documentation for PMS logic
- **Battery integration**: Add energy storage to the system
- **Shore power**: Check `02_Shore_Power_Example.ipynb` for emission reduction strategies
- **Custom load profiles**: Use actual operational data instead of random profiles

**Integration opportunities**:
- Connect to voyage planning tools
- Link with vessel monitoring systems
- Automate regulatory reporting
- Support digital twin applications